In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Let's see the size of the distribution of the data.

In [ ]:
print("Size of Spam Data:", len(os.listdir('/kaggle/input/ham-and-spam-dataset/hamnspam/spam/')))
print("Size of Ham Data:", len(os.listdir('/kaggle/input/ham-and-spam-dataset/hamnspam/ham/')))

# Preparing the raw data and convert into DataFrame format.

In [ ]:
path = '/kaggle/input/ham-and-spam-dataset/hamnspam/'
mails = []
labels = []

for label in ['ham/', 'spam/']:
#     filenames = os.listdir(os.path.join(path, label))
    filenames = os.listdir(path + label)
    for file in filenames:
        f = open((path + label + file), 'r', encoding = 'latin-1')
        bolk = f.read()
        mails.append(bolk)
        labels.append(label)
        
df = pd.DataFrame({'emails': mails, 'labels': labels})

# Sample:

In [ ]:
mails[1]

In [ ]:
labels[1]

In [ ]:
df

# Converting the labels 'ham/' and 'spam/' to 0 and 1.

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['labels'] = encoder.fit_transform(df['labels'])

# Let's see how many NaNs are there to spoil the data...

In [ ]:
df.isnull().sum()

# Converting the data into lower case.

In [ ]:
df['emails'] = df['emails'].apply(lambda x:x.lower())

In [ ]:
df

# Replacing '\n' and '\t' with ' ' for cleaning.

In [ ]:
df['emails'] = df['emails'].apply(lambda x: x.replace('\n', ' '))
df['emails'] = df['emails'].apply(lambda x: x.replace('\t', ' '))

In [ ]:
df

In [ ]:
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

## Defining Porter Stemmer for future use.

In [ ]:
ps = PorterStemmer()

# Corpus is where all the processed data will be stored.

In [ ]:
corpus = []

# #1 Removing all special charactors and numbers
# #2 Using Stemming here to reduce the data and make more sensible.


# Wait.....

In [ ]:
for i in range(len(df)):
    
    ## Becuase of removed duplicated entries...
    
    review = re.sub('[^a-zA-Z]', ' ', df['emails'][i])
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

# Using Bag of Words approach for Data Preparation.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2500)
X = cv.fit_transform(corpus).toarray()
y = df['labels']

# Splitting the Data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Using MultiNomial Naive Bayes as the Model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

# Check Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

# Check the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
CM = confusion_matrix(y_test, y_pred)

In [ ]:
CM